In [ ]:
from utils.edhec import StrategyEngine, GoalPriceIndex
from utils.data import YieldCurveBuilder, DataLoader
from utils.constant import INFLATION_SALAIRE, RETIREMENT_DATE ,ASSET_FILE ,YIELD_FILE,CLIENT_AGE_START, INITIAL_WEALTH, INITIAL_SALARY, SAVINGS_RATE, CONTRIB_START, AGE_PEAK, CONTRIB_PEAK, SIMULATION_START, SIMULATION_END, FLOOR_PERCENT
import pandas as pd
import matplotlib.pyplot as plt
from utils.helpers import print_simulation_summary

In [ ]:

yc = YieldCurveBuilder().load_from_csv(YIELD_FILE)
assets = DataLoader.get_asset_returns(ASSET_FILE)

if yc and assets is not None:
    gpi = GoalPriceIndex(yc, RETIREMENT_DATE)
    
    # 2. Backtest
    engine1 = StrategyEngine(gpi, assets['US Equity USD Unhedged'], assets['US Government Bond USD Unhedged'],
                            SIMULATION_START, SIMULATION_END, INITIAL_WEALTH, INITIAL_SALARY, SAVINGS_RATE, INFLATION_SALAIRE)
    engine2 = StrategyEngine(gpi, assets['US Equity USD Unhedged'], assets['US Government Bond USD Unhedged'],
                            SIMULATION_START, SIMULATION_END, INITIAL_WEALTH, INITIAL_SALARY, SAVINGS_RATE, INFLATION_SALAIRE)
    
    gbi_res = engine1.run_gbi(FLOOR_PERCENT)
    tdf_res = engine2.run_tdf()
    


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gbi_res.index, gbi_res['Wealth'], label='GBI Wealth', linewidth=2, color='orange')
plt.plot(gbi_res.index, gbi_res['Floor'], label='GBI Floor', linewidth=2, color='red')
plt.plot(tdf_res.index, tdf_res['Wealth'], label='TDF Wealth', linewidth=2, color='blue')
plt.plot(gbi_res.index, gbi_res['Contrib_mensuel'].cumsum() + INITIAL_WEALTH, label='Wealth invest', linewidth=2, color='green')
plt.xlabel('Date')
plt.ylabel('Wealth')
plt.title('Wealth Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.fill_between(gbi_res.index, 0, gbi_res['Allocation_PSP']*100, label='Actions',color='#F18F01', alpha=0.5)
plt.fill_between(gbi_res.index, gbi_res['Allocation_PSP']*100, 100, color='#006BA6', alpha=0.5, label='Obligations')
plt.plot(tdf_res.index, tdf_res['Allocation_PSP']*100, label='TDF Actions',color="#01F10D", alpha=0.3, linewidth=4)
plt.xlabel('Date')
plt.ylabel('PSP Allocation')
plt.title('Gbi PSP Allocation Over Time')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print_simulation_summary(gbi_res, "GBI (Goal-Based Investing)", INITIAL_WEALTH)
print_simulation_summary(tdf_res, "TDF (Target Date Fund)", INITIAL_WEALTH)